In [1]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed
import numpy as np
import cv2
import os
import sys

# Import matplotlib libraries
from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection
import matplotlib.patches as patches

# Some modules to display an animation using imageio.
import imageio
from IPython.display import HTML, display
import pandas as pd
# import utils

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed
import numpy as np
import cv2

# Import matplotlib libraries
from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection
import matplotlib.patches as patches

# Some modules to display an animation using imageio.
import imageio
from IPython.display import HTML, display

In [3]:
model_name = "movenet_lightning"

if "tflite" in model_name:
  if "movenet_lightning_f16" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/lightning/tflite/float16/4?lite-format=tflite
    input_size = 192
  elif "movenet_thunder_f16" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/thunder/tflite/float16/4?lite-format=tflite
    input_size = 256
  elif "movenet_lightning_int8" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/lightning/tflite/int8/4?lite-format=tflite
    input_size = 192
  elif "movenet_thunder_int8" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/thunder/tflite/int8/4?lite-format=tflite
    input_size = 256
  else:
    raise ValueError("Unsupported model name: %s" % model_name)

  # Initialize the TFLite interpreter
  interpreter = tf.lite.Interpreter(model_path="model.tflite")
  interpreter.allocate_tensors()

  def movenet(input_image):
    """Runs detection on an input image.

    Args:
      input_image: A [1, height, width, 3] tensor represents the input image
        pixels. Note that the height/width should already be resized and match the
        expected input resolution of the model before passing into this function.

    Returns:
      A [1, 1, 17, 3] float numpy array representing the predicted keypoint
      coordinates and scores.
    """
    # TF Lite format expects tensor type of uint8.
    input_image = tf.cast(input_image, dtype=tf.uint8)
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    interpreter.set_tensor(input_details[0]['index'], input_image.numpy())
    # Invoke inference.
    interpreter.invoke()
    # Get the model prediction.
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    return keypoints_with_scores

else:
  if "movenet_lightning" in model_name:
    module = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
    input_size = 192
  elif "movenet_thunder" in model_name:
    module = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/4")
    input_size = 256
  else:
    raise ValueError("Unsupported model name: %s" % model_name)

  def movenet(input_image):
    """Runs detection on an input image.

    Args:
      input_image: A [1, height, width, 3] tensor represents the input image
        pixels. Note that the height/width should already be resized and match the
        expected input resolution of the model before passing into this function.

    Returns:
      A [1, 1, 17, 3] float numpy array representing the predicted keypoint
      coordinates and scores.
    """
    model = module.signatures['serving_default']

    # SavedModel format expects tensor type of int32.
    input_image = tf.cast(input_image, dtype=tf.int32)
    # Run model inference.
    outputs = model(input_image)
    # Output is a [1, 1, 17, 3] tensor.
    keypoints_with_scores = outputs['output_0'].numpy()
    return keypoints_with_scores

In [4]:
# Load the input image.
image_path = r'D:\Pemrograman\Python\Project\Hidup-Sehat-Machine-Learning\PoseEstimation\images\stress_relief\Easy-Pose_Andrew-Clark_1.jpg'
image = tf.io.read_file(image_path)
image = tf.image.decode_jpeg(image)

In [5]:
# %pip install -q git+https://github.com/tensorflow/docs

In [14]:
import tensorflow

In [15]:
def draw_prediction_on_image(
    image, person, crop_region=None, close_figure=True,
    keep_input_size=False):
  """Draws the keypoint predictions on image.
 
  Args:
    image: An numpy array with shape [height, width, channel] representing the
      pixel values of the input image.
    person: A person entity returned from the MoveNet.SinglePose model.
    close_figure: Whether to close the plt figure after the function returns.
    keep_input_size: Whether to keep the size of the input image.
 
  Returns:
    An numpy array with shape [out_height, out_width, channel] representing the
    image overlaid with keypoint predictions.
  """
  # Draw the detection result on top of the image.
  image_np = utils.visualize(image, [person])
  
  # Plot the image with detection results.
  height, width, channel = image.shape
  aspect_ratio = float(width) / height
  fig, ax = plt.subplots(figsize=(12 * aspect_ratio, 12))
  im = ax.imshow(image_np)
 
  if close_figure:
    plt.close(fig)
 
  if not keep_input_size:
    image_np = utils.keep_aspect_ratio_resizer(image_np, (512, 512))

  return image_np

In [16]:
# Resize and pad the image to keep the aspect ratio and fit the expected size.
input_image = tf.expand_dims(image, axis=0)
input_image = tf.image.resize_with_pad(input_image, input_size, input_size)

# Run model inference.
keypoints_with_scores = movenet(input_image)

# Visualize the predictions with image.
display_image = tf.expand_dims(image, axis=0)
display_image = tf.cast(tf.image.resize_with_pad(
    display_image, 1280, 1280), dtype=tf.int32)
output_overlay = draw_prediction_on_image(
    np.squeeze(display_image.numpy(), axis=0), keypoints_with_scores)

plt.figure(figsize=(5, 5))
plt.imshow(output_overlay)
_ = plt.axis('off')

AttributeError: module 'keras.utils' has no attribute 'visualize'

In [20]:
# # Import TF and TF Hub libraries.
# import tensorflow as tf
# import tensorflow_hub as hub

# # Load the input image.
# image_path = r'D:\Pemrograman\Python\Project\Pose Estimation\poses_images_out_train\adho mukha svanasana\1. 1.png'
# image = tf.io.read_file(image_path)
# image = tf.compat.v1.image.decode_jpeg(image)[:,:,:3]
# image = tf.expand_dims(image, axis=0)
# # Resize and pad the image to keep the aspect ratio and fit the expected size.
# image = tf.cast(tf.image.resize_with_pad(image, 192, 192), dtype=tf.int32)

# # Download the model from TF Hub.
# model = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
# movenet = model.signatures['serving_default']

# # Run model inference.
# outputs = movenet(image)
# # Output is a [1, 1, 17, 3] tensor.
# keypoints = outputs['output_0']


In [21]:
# # 17 keypoints in the model output
# kp_descriptions = [
#     'nose', 'left_eye', 'right_eye', 'left_ear', 'right_ear',
#     'left_shoulder', 'right_shoulder', 'left_elbow', 'right_elbow',
#     'left_wrist', 'right_wrist', 'left_hip', 'right_hip',
#     'left_knee', 'right_knee', 'left_ankle', 'right_ankle'
# ]
# # Rename columns in the DataFrames according to the values
# columns = []
# for point in kp_descriptions:
#     for value in ('y', 'x', 'score'):
#         columns.append(f'{point}_{value}')

In [22]:
# pd.DataFrame(np.array(keypoints[0][0]).flatten().reshape(1,-1),columns = columns)

In [11]:
# # import required libraries
# import numpy as np
# from numpy.linalg import norm
 
# # define two lists or array
# A = np.array([0.58076525, 0.42730328, 0.67503864, 0.5792289 , 0.41207743,
#        0.47333997, 0.58185965, 0.41216052, 0.52444524, 0.5445528 ,
#        0.39484406, 0.45617175, 0.5446289 , 0.3936121 , 0.46745512,
#        0.5016635 , 0.4046818 , 0.7063899 , 0.49563456, 0.40258092,
#        0.54718405, 0.60904396, 0.34574008, 0.7440503 , 0.5912181 ,
#        0.34356624, 0.6541515 , 0.70214546, 0.26372153, 0.9110583 ,
#        0.6600009 , 0.27307063, 0.38330933, 0.32035175, 0.6203919 ,
#        0.61942005, 0.32266206, 0.60961163, 0.76251876, 0.4952395 ,
#        0.7256286 , 0.6774133 , 0.4977281 , 0.70222014, 0.80471295,
#        0.6468238 , 0.8216442 , 0.75596136, 0.62307787, 0.78378725,
#        0.60452706])

# B = np.array([0.5749743 , 0.42486113, 0.54318196, 0.57792485, 0.40930325,
#        0.5496756 , 0.5782155 , 0.40489376, 0.57750416, 0.5340253 ,
#        0.38575894, 0.5966261 , 0.5361736 , 0.37668896, 0.54088706,
#        0.50579745, 0.43160397, 0.6138604 , 0.48600656, 0.37216204,
#        0.6575917 , 0.63704824, 0.33847758, 0.64200926, 0.587188  ,
#        0.31143132, 0.7014244 , 0.72440684, 0.2353007 , 0.6007997 ,
#        0.6784536 , 0.23021728, 0.51602674, 0.30333444, 0.6349661 ,
#        0.74985135, 0.31460917, 0.60921407, 0.68592465, 0.47325823,
#        0.77171266, 0.6738131 , 0.48240095, 0.73322797, 0.8058868 ,
#        0.6358363 , 0.84106433, 0.6906332 , 0.6143594 , 0.7967786 ,
#        0.7987496 ])
 
# print("A:", A)
# print("B:", B)
 
# # compute cosine similarity
# cosine = np.dot(A,B)/(norm(A)*norm(B))
# print("Cosine Similarity:", cosine)

A: [0.58076525 0.42730328 0.67503864 0.5792289  0.41207743 0.47333997
 0.58185965 0.41216052 0.52444524 0.5445528  0.39484406 0.45617175
 0.5446289  0.3936121  0.46745512 0.5016635  0.4046818  0.7063899
 0.49563456 0.40258092 0.54718405 0.60904396 0.34574008 0.7440503
 0.5912181  0.34356624 0.6541515  0.70214546 0.26372153 0.9110583
 0.6600009  0.27307063 0.38330933 0.32035175 0.6203919  0.61942005
 0.32266206 0.60961163 0.76251876 0.4952395  0.7256286  0.6774133
 0.4977281  0.70222014 0.80471295 0.6468238  0.8216442  0.75596136
 0.62307787 0.78378725 0.60452706]
B: [0.5749743  0.42486113 0.54318196 0.57792485 0.40930325 0.5496756
 0.5782155  0.40489376 0.57750416 0.5340253  0.38575894 0.5966261
 0.5361736  0.37668896 0.54088706 0.50579745 0.43160397 0.6138604
 0.48600656 0.37216204 0.6575917  0.63704824 0.33847758 0.64200926
 0.587188   0.31143132 0.7014244  0.72440684 0.2353007  0.6007997
 0.6784536  0.23021728 0.51602674 0.30333444 0.6349661  0.74985135
 0.31460917 0.60921407 0.6859

In [7]:
# np.array(keypoints[0][0]).flatten().reshape(1,-1)

array([[0.58076525, 0.42730328, 0.67503864, 0.5792289 , 0.41207743,
        0.47333997, 0.58185965, 0.41216052, 0.52444524, 0.5445528 ,
        0.39484406, 0.45617175, 0.5446289 , 0.3936121 , 0.46745512,
        0.5016635 , 0.4046818 , 0.7063899 , 0.49563456, 0.40258092,
        0.54718405, 0.60904396, 0.34574008, 0.7440503 , 0.5912181 ,
        0.34356624, 0.6541515 , 0.70214546, 0.26372153, 0.9110583 ,
        0.6600009 , 0.27307063, 0.38330933, 0.32035175, 0.6203919 ,
        0.61942005, 0.32266206, 0.60961163, 0.76251876, 0.4952395 ,
        0.7256286 , 0.6774133 , 0.4977281 , 0.70222014, 0.80471295,
        0.6468238 , 0.8216442 , 0.75596136, 0.62307787, 0.78378725,
        0.60452706]], dtype=float32)